<a href="https://colab.research.google.com/github/mtrrn1016/SC/blob/main/%EB%A8%B8%EB%9E%AD%EC%9D%B4%EB%84%A4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#작업

## 코드 설계 계획

### **주제 : 반려동물 동반시설 추천 프로그램**

---


**- 활용할 데이터**

공공데이터포털 - 한국정보원_전국 반려동물 동반 가능 문화시설 위치 데이터
[링크 텍스트](https://www.data.go.kr/data/15111389/fileData.do)

---


**- 프로그램 설명**

프로그램 명 : '머랭이네 반려동물 시설 추천'

사용자의 입력 정보에 맞는 반려동물 시설 추천프로그램.

각 항목이 사용자 입력정보에 맞는지 분석, 해당할수록 낮은 점수 부여.

100점 이하의 시설을 점수가 낮은 순으로 최대 10개까지 리스트로 출력해 추천.

추천 받고 나면 프로그램을 종료할 것인지 재추천 받을 것인지 선택, 종료하면 프로그램 종료라고 재추천받고 싶다면 사용자에게 입력정보부터 재입력 받아 추천.

(시설 유형과 같이 아예 제외 시켜야하는 항목에는 점수를 초과할 수 밖에 없는 점수를 주어 배제)

---


**- 입력받을 정보**

사용자 위치 정보(시/도, 시/군/구, 동)

시간 정보

시설 유형

주차

실내/외 선호 여부

---


**⦁	점수 부여 기능**

- 시설 유형
엑셀의 카테고리3 항목 분석해 일치하지 않는 시설 유형은 배제

- 시간 정보
사용자가 입력한 시간대가 운영시간대에 포함된다면 0점, 정보가 없다면 10점, 포함되지 않는다면 1000점을 부여

**문제 발생** :
데이터 파일의 시간 정보가 '월~금 09:00~18:00' 의 복잡한 형태로 바로 인식 및 분류 불가능 >>> 요일과 시간을 분리, 정보에 따른 반환 및 분류 필요

- 위치
같은 동은 0점, 다른 동은 1점, 같은 구는 0점, 다른 구는 10점, 같은 시는 0점, 다른 시는 1000점으로 차등 점수 부여

- 주차여부
주차가 가능하면 0점, 불가능은 10점

- 실내/외 선호도
실내/외 선호정보와 같은 정보의 항목엔 0점, 다른 경우 1000점 부여

---

**⦁	출력 기능**

최대 10개의 적합한 시설들을 점수가 낮은 순으로 정렬해 리스트로 출력
- 출력할 항목 (괄호 안 항목은 데이터파일의 해당 열)
시설명(A), 시설유형(D), 위치정보(O or P), 전화번호(Q), 홈페이지(R), 운영시간(T), 주차가능(U), 실내/외(AA or AB)

### **- 작업단계**

1. 데이터 로드 및 사전 처리
2. 운영시간 항목 분류 및 반환처리
3. 사용자 입력 정보 획득
4. 시설 점수 계산 기능 구현
5. 추천 프로그램 실행 로직 구현
**굵은 텍스트**



## 코드 단계 별 설명

### 데이터 로드 및 사전 처리

데이터 파일 로드와 중복 시설 배제, 프로그램명을 정의한다.


**추론**:
data.csv 파일을 Pandas DataFrame으로 로드하고, 중복되는 동일시설을 지우기 위해 시설유형과 지번주소가 같은 시설을 배제시킨다. 프로그램명을 정의한다.

### 운영시간 항목 분류 및 반환처리



**추론**:
복잡한 '운영시간' 열을 구문 분석하고 현재 시간이 시설의 운영 시간에 속하는지 확인하려면 도우미 함수 'is_inhowes'를 구현해야 합니다. 이 함수는 메인 'calcate_score' 함수를 구현하기 전에 '24 시간' 또는 '변동'와 같은 다양한 일 형식, 시간 범위 및 특수 문자열을 처리합니다.



 **'운영시간' 열을 구문 분석할 때 어떤 설계 고려 사항**

    '운영시간' 열은 복잡하고 다양한 문자열 패턴 세트를 보여준다. 이를 처리하려면 유틸리티 함수가 필요하다:
    *   다양한 요일 사양(예: 특정 요일, '월~금' 또는 '매일'와 같은 요일 범위).
    *   쉼표 또는 새 줄로 구분된 여러 항목.
    *   시간 형식은 'HH:MM~HH:MM'같은 형태.
    *   자정을 넘는 시간 범위(예: '22:00~02:00').
    *   '24 시간', '상시', '변동', '정보 없음'와 같은 특별 키워드.
    *   공휴일 포함 ('법정공휴일').


### 사용자 입력 정보 획득

사용자로부터 시설유형, 도착 예정 시간, 출발 위치, 주차 선호도, 실내/외 선호도를 입력받는 함수를 구현하고, 각 입력 항목에 대한 유효성 검사 및 안내를 한다.


**추론**:

사용자 선호도를 확인할 수 있는 기능을 구현한다. 먼저 'get_user_preferences' 함수를 정의하고 첫 번째 입력인 시설 유형('카테고리3')부터 구현. 여기에는 DataFrame에서 고유한 카테고리를 검색하여 사용자에게 표시하고 입력을 검증하는 작업이 포함된다.

'get_user_preferences' 기능을 계속해서 구현하면서, 사용자의 현재 위치를 획득한다. 이는 각 레벨에서 동적 옵션과 검증을 통해 '시도 명칭', '시군구 명칭', '법정읍면동명칭'를 묻는 프롬프트를 포함한다.

주차 선호도('주차 가능여부')와 실내/외 선호도('장소(실내) 여부', '장소(실외) 여부') 입력을 검증과 함께 구현하여 작업 지침에 따라 'get_user_preferences' 기능을 완료한다.


**사용자 선호도 입력 받기 (get_user_preferences 호출):**

* preferences = get_user_preferences(df.copy()) : 이전에 구현된 get_user_preferences 함수를 호출하여 사용자가 원하는 시설 유형, 도착 시간, 위치, 주차 선호도, 실내/외 선호도 등의 정보를 입력받는다.
* df.copy()로 넘겨주는 것 = 원본 DataFrame이 get_user_preferences 내부에서 변경되지 않도록 하기 위해서이다.

### 시설 점수 계산 기능 구현

각 시설에 대해 사용자 입력 조건에 따른 점수를 계산하는 `calculate_score` 함수를 구현한다.


**추론**:
주요 '계산_점수' 함수를 구현한다. 획득한 시설 행과 사용자 선호도로 시설 유형, 운영 시간, 위치, 주차 및 실내/외 선호도에 따라 점수를 계산하고 합산한다.



**각 시설 점수 계산 (calculate_score 적용):**

* df['score'] = df.apply(lambda row: calculate_score(row, preferences), axis=1) : DataFrame의 각 시설(row)에 대해 calculate_score 함수를 적용해 추천 점수를 계산한다.
* axis=1 : 행(row) 단위로 함수를 적용.
* 계산된 점수는 score라는 새로운 컬럼에 저장된다. 점수가 낮을수록 사용자 선호도에 더 잘 맞는 시설이다.
* print("사용자 선호도에 따라 모든 시설의 점수 계산 완료.") : 점수 계산이 완료되었음을 사용자에게 알린다.

### 추천 프로그램 실행 로직 구현

메인 추천 프로그램(`recommendation_program`) 함수를 구현하고, 사용자 입력, 점수 계산, 필터링, 정렬, 추천 목록 출력, 그리고 재추천/종료 기능을 포함하는 무한 루프 로직을 완성한다.


**추론**:
주요 '추천_프로그램' 기능을 구현한다. 함수를 정의해 무한 루프를 시작한다.  'get_user_preferences'호출로 사용자에게 입력을 받아 'calculate_score' 함수로 각 시설의 점수를 계산한다.

100점 이하의 시설을 필터링하고 낮은 점수 순으로 정렬한 다음 상위 10개를 추출해 결과를 표시한다. 사용자에게 프로그램을 다시 추천하거나 종료하라는 메시지를 표시하여 루프 로직을 완성한다.


**프로그램 시작 및 무한 루프 (while True):**

* print(f"\n--- {program_name} 시작 ---") : 프로그램 시작 메시지를 출력한다.

* while True: : 사용자가 프로그램을 종료하기 전까지 계속해서 추천을 받을 수 있도록 무한 루프를 시작한다. 사용자가 'Q'를 입력하면 루프를 종료한다.



**추천 시설 필터링 및 정렬:**

* recommended_facilities = df[df['score'] <= 100] : 계산된 점수가 100점 이하인 시설들만 필터링. (100점 초과는 추천에서 사실상 배제)
* recommended_facilities = recommended_facilities.sort_values(by='score').reset_index(drop=True) : 필터링된 시설들을 score 컬럼을 기준으로 오름차순(점수가 낮은 순)으로 정렬한다.
* top_10_recommendations = recommended_facilities.head(10) : 정렬된 시설들 중에서 가장 점수가 낮은 상위 10개 시설을 추출한다.

**추천 목록 출력:**

* print("\n--- 추천 시설 목록 ---") : 추천 목록의 시작을 알리는 메시지를 출력한다.
* if not top_10_recommendations.empty: : 추천된 시설이 있는 경우에만 추천 시설을 출력한다.
* for i, row in top_10_recommendations.iterrows(): : 상위 10개 추천 시설 각각에 대해 반복하여 상세 정보를 출력한다.

* 출력되는 정보 = [시설명, 카테고리, 주소, 전화번호, 홈페이지, 운영시간, 주차 가능 여부, 실내/외 여부, 계산된 추천 점수]
* else: : 추천된 시설이 없는 경우, "현재 조건에 맞는 추천 시설을 찾을 수 없습니다." 메시지를 출력한다.

**재추천 또는 종료 선택:**

* while True:와 action = input("\n다시 추천받으시겠습니까? (R: 다시 추천, Q: 종료): ").strip().upper(): 사용자에게 재추천을 받을지 프로그램을 종료할지 선택하도록 한다.
* if action == 'R': break : 'R'을 입력하면 현재 내부 루프를 빠져나가 while True (프로그램 시작) 루프의 처음으로 돌아간다.
* elif action == 'Q': print("프로그램을 종료합니다. 감사합니다!"); return : 'Q'를 입력하면 종료 메시지를 출력한다. return을 통해 recommendation_program 함수 전체를 종료한다.
* else: : 유효하지 않은 입력을 했을 경우 다시 입력 요청 메시지를 출력한다.

## 프로그램의 최종 구현 결과와 작동 방식 요약 및 설명


1.   data.csv 파일 로드 및 필수 컬럼의 데이터 품질 확인

2.   사용자 선호도(시설 유형, 현재 시간, 위치, 주차, 실내/외 선호)를 입력받는 get_user_preferences 함수 구현

3. 운영시간 문자열을 파싱하고 현재 운영 여부를 판단하는 is_within_operating_hours 헬퍼 함수 개발

4. 사용자 선호도에 따라 시설에 점수를 부여하는 calculate_score 함수 구현

5. 사용자 입력에 따라 시설을 필터링, 정렬하고 상위 10개를 추천하며, 재추천 또는 종료 옵션을 제공하는 메인 recommendation_program 로직 완성


## 2단계 시 고려 사항과 데이터 분석 결과

# 프로그램 코드

In [3]:
# 데이터 로드 및 사전 처리
import pandas as pd
import re
from datetime import datetime, time

# 데이터 프레임 속 data.csv 로드하기
df = pd.read_csv('/content/data.csv')

# '시설명'과 '지번주소'가 겹치는 동일 시설 제외하기
df = df.drop_duplicates(subset=['시설명', '지번주소']).reset_index(drop=True)

# 프로그램명 정의
program_name = '머랭이네 반려동물 시설 추천'

In [4]:
# 운영시간항목 분류 및 반환처리
def is_within_operating_hours(operating_hours_str, current_time_str):
    """
    운영시간 문자열과 현재 시간을 받아 현재 시간이 운영시간에 포함되는지 여부를 반환.
    운영시간 정보가 없거나 '변동' 등 파싱 불가능한 경우 None을 반환.
    """
    if pd.isna(operating_hours_str) or not operating_hours_str.strip():
        return None # 운영시간 정보 없음

    operating_hours_str = operating_hours_str.strip()

    # 특수 케이스 처리
    if "24시간" in operating_hours_str or "상시" in operating_hours_str:
        return True
    if "변동" in operating_hours_str or "정보 없음" in operating_hours_str:
        return None # 파싱 불가능한 정보로 간주

    # 현재 요일과 시간 준비
    # '월', '화', '수', '목', '금', '토', '일' 인덱싱을 위해 0부터 6까지 매핑
    day_mapping = {'월': 0, '화': 1, '수': 2, '목': 3, '금': 4, '토': 5, '일': 6}
    today_weekday_korean = ['월', '화', '수', '목', '금', '토', '일'][datetime.now().weekday()]
    current_time_obj = datetime.strptime(current_time_str, '%H:%M').time()

    segments = operating_hours_str.split(',')
    for segment in segments:
        segment = segment.strip()
        if not segment: continue

        # 정규식 패턴: 요일 부분 (한글 요일, ~ 포함) + 시간 부분 (HH:MM~HH:MM)
        match = re.match(r'([가-힣,~\s]+)\s*(\d{2}:\d{2}~\d{2}:\d{2})', segment)

        if match:
            day_part = match.group(1).strip()
            time_range_str = match.group(2).strip()

            # 요일 파싱
            is_today_in_day_part = False
            if '~' in day_part: # 예: 월~금
                day_range_parts = day_part.split('~')
                if len(day_range_parts) == 2: # 유효한 범위인지 확인 (정확히 두 부분)
                    start_day_korean, end_day_korean = day_range_parts
                    start_idx = day_mapping.get(start_day_korean)
                    end_idx = day_mapping.get(end_day_korean)
                    if start_idx is not None and end_idx is not None:
                        if start_idx <= end_idx:
                            valid_weekdays = [v for k,v in day_mapping.items() if start_idx <= v <= end_idx]
                        else: # 예: 수~월 (수,목,금,토,일,월)
                            valid_weekdays = [v for k,v in day_mapping.items() if v >= start_idx or v <= end_idx]
                        if day_mapping[today_weekday_korean] in valid_weekdays:
                            is_today_in_day_part = True
            elif '매일' in day_part:
                is_today_in_day_part = True
            else: # 예: 월, 월,목
                # 여러 구분자 (공백, 쉼표)를 사용하여 요일 분리
                days = re.split(r'[, ]+', day_part)
                if today_weekday_korean in days:
                    is_today_in_day_part = True

            if is_today_in_day_part:
                # 시간 범위 파싱
                try:
                    start_time_str, end_time_str = time_range_str.split('~')
                    start_time_obj = datetime.strptime(start_time_str, '%H:%M').time()
                    end_time_obj = datetime.strptime(end_time_str, '%H:%M').time()

                    # 현재 시간이 운영 시간 내에 있는지 확인
                    if start_time_obj <= end_time_obj:
                        if start_time_obj <= current_time_obj < end_time_obj:
                            return True
                    else: # 운영 시간이 자정을 넘어가는 경우 (예: 22:00~02:00)
                        if current_time_obj >= start_time_obj or current_time_obj < end_time_obj:
                            return True
                except ValueError: # 시간 파싱 오류
                    continue

    return False # 해당 요일에 운영 시간이 없거나 현재 시간이 운영 시간 외

In [5]:
#사용자 입력 정보 획득
def get_user_preferences(df):
    """
    사용자로부터 시설유형, 현재 시간, 현재 위치, 주차 선호도, 실내/외 선호도를 입력받아 딕셔너리 형태로 반환합니다.
    각 입력 항목에 대한 유효성 검사 및 안내를 포함합니다.
    """
    preferences = {}

    # 1. 시설유형 (카테고리3) 입력받기
    unique_categories = df['카테고리3'].dropna().unique().tolist()
    print("\n--- 시설유형 선택 ---")
    print("선택 가능한 시설유형:")
    for i, category in enumerate(unique_categories):
        print(f"  {i+1}. {category}")

    while True:
        try:
            category_input = input("원하는 시설유형을 입력하세요 (번호 또는 이름): ").strip()
            if category_input.isdigit():
                idx = int(category_input) - 1
                if 0 <= idx < len(unique_categories):
                    preferences['시설유형'] = unique_categories[idx]
                    break
                else:
                    print("유효하지 않은 번호입니다. 다시 입력해주세요.")
            elif category_input in unique_categories:
                preferences['시설유형'] = category_input
                break
            else:
                print("유효하지 않은 시설유형입니다. 목록에서 선택하거나 정확히 입력해주세요.")
        except Exception as e:
            print(f"입력 오류: {e}. 다시 시도해주세요.")

    # 2. 도착 예정 시간 입력받기 (HH:MM 형식)
    print("\n--- 도착 예정 시간 입력 ---")
    while True:
        time_str = input("도착 예정 시간을 'HH:MM' 형식으로 입력하세요 (예: 09:30, 14:00): ").strip()
        if re.match(r'^([01]\d|2[0-3]):([0-5]\d)$', time_str):
            preferences['도착 시간'] = time_str
            break
        else:
            print("잘못된 시간 형식입니다. 'HH:MM' 형식으로 다시 입력해주세요.")

    # 3. 출발 위치 입력받기 (시도 명칭, 시군구 명칭, 읍면동명칭)
    print("\n--- 출발 위치 입력 ---")
    start_df = df.copy()

    # 시도 명칭 입력
    while True:
        unique_sido = start_df['시도 명칭'].dropna().unique().tolist()
        print("\n선택 가능한 시도 명칭:")
        for i, sido in enumerate(unique_sido):
            print(f"  {i+1}. {sido}")
        sido_input = input("시도 명칭을 입력하세요 (번호 또는 이름): ").strip()

        selected_sido = None
        if sido_input.isdigit():
            idx = int(sido_input) - 1
            if 0 <= idx < len(unique_sido):
                selected_sido = unique_sido[idx]
        elif sido_input in unique_sido:
            selected_sido = sido_input

        if selected_sido:
            preferences['시도 명칭'] = selected_sido
            start_df = start_df[start_df['시도 명칭'] == selected_sido]
            break
        else:
            print("유효하지 않은 시도 명칭입니다. 다시 입력해주세요.")

    # 시군구 명칭 입력
    while True:
        unique_sigungu = start_df['시군구 명칭'].dropna().unique().tolist()
        if not unique_sigungu:
            print("선택 가능한 시군구 명칭이 없습니다.")
            preferences['시군구 명칭'] = None
            break
        print(f"\n'{selected_sido}' 내 선택 가능한 시군구 명칭:")
        for i, sigungu in enumerate(unique_sigungu):
            print(f"  {i+1}. {sigungu}")
        sigungu_input = input("시군구 명칭을 입력하세요 (번호 또는 이름): ").strip()

        selected_sigungu = None
        if sigungu_input.isdigit():
            idx = int(sigungu_input) - 1
            if 0 <= idx < len(unique_sigungu):
                selected_sigungu = unique_sigungu[idx]
        elif sigungu_input in unique_sigungu:
            selected_sigungu = sigungu_input

        if selected_sigungu:
            preferences['시군구 명칭'] = selected_sigungu
            start_df = start_df[start_df['시군구 명칭'] == selected_sigungu]
            break
        else:
            print("유효하지 않은 시군구 명칭입니다. 다시 입력해주세요.")

    # 읍면동명칭 입력
    while True:
        unique_eupmyeondong = start_df['법정읍면동명칭'].dropna().unique().tolist()
        if not unique_eupmyeondong:
            print("선택 가능한 읍면동 명칭이 없습니다.")
            preferences['법정읍면동명칭'] = None
            break
        print(f"\n'{selected_sigungu}' 내 선택 가능한 읍면동 명칭:")
        for i, eupmyeondong in enumerate(unique_eupmyeondong):
            print(f"  {i+1}. {eupmyeondong}")
        eupmyeondong_input = input("읍면동 명칭을 입력하세요 (번호 또는 이름): ").strip()

        selected_eupmyeondong = None
        if eupmyeondong_input.isdigit():
            idx = int(eupmyeondong_input) - 1
            if 0 <= idx < len(unique_eupmyeondong):
                selected_eupmyeondong = unique_eupmyeondong[idx]
        elif eupmyeondong_input in unique_eupmyeondong:
            selected_eupmyeondong = eupmyeondong_input

        if selected_eupmyeondong:
            preferences['법정읍면동명칭'] = selected_eupmyeondong
            break
        else:
            print("유효하지 않은 읍면동 명칭입니다. 다시 입력해주세요.")

    # 4. 주차 가능 여부 선호도 입력받기
    print("\n--- 주차 선호도 ---")
    while True:
        parking_input = input("주차 가능 여부 선호도를 입력하세요 (Y: 예, N: 아니오): ").strip().upper()
        if parking_input in ['Y', 'N']:
            preferences['주차 가능여부'] = parking_input
            break
        else:
            print("유효하지 않은 입력입니다. 'Y' 또는 'N'으로 다시 입력해주세요.")

    # 5. 장소의 실내/외 선호도 입력받기
    print("\n--- 장소(실내/실외) 선호도 ---")
    while True:
        location_type_input = input("장소 선호도를 입력하세요 (실내, 실외, 무관): ").strip()
        if location_type_input in ['실내', '실외', '무관']:
            preferences['장소 선호도'] = location_type_input
            break
        else:
            print("유효하지 않은 입력입니다. '실내', '실외', '무관' 중 하나로 다시 입력해주세요.")

    print(f"\n입력한 내용 : {preferences}")
    return preferences

In [10]:
# 시설 점수 계산 기능 구현
def calculate_score(facility_row, preferences):
    """
    각 시설에 대해 사용자 입력 조건에 따른 점수를 계산합니다.
    점수가 낮을수록 사용자 선호도에 부합합니다.
    """
    score = 0

    # 1. 시설 유형 (카테고리3) 점수 계산
    if facility_row['카테고리3'] == preferences['시설유형']:
        score += 0
    else:
        score += 1000 # 불일치 시 높은 점수 부여 (추천에서 배제)

    # 2. 시간 정보 (운영시간) 점수 계산
    operating_status = is_within_operating_hours(facility_row['운영시간'], preferences['도착 시간'])
    if operating_status is None:
        score += 10 # 운영시간 정보 없음 또는 파싱 불가능한 경우 (감점)
    elif operating_status:
        score += 0 # 현재 운영 중인 경우
    else:
        score += 1000 # 현재 운영 시간이 아닌 경우 (추천에서 배제)

    # 3. 위치 점수 계산
    user_sido = preferences.get('시도 명칭')
    user_sigungu = preferences.get('시군구 명칭')
    user_eupmyeondong = preferences.get('법정읍면동명칭')

    facility_sido = facility_row.get('시도 명칭')
    facility_sigungu = facility_row.get('시군구 명칭')
    facility_eupmyeondong = facility_row.get('법정읍면동명칭')

    if facility_sido != user_sido:
        score += 1000 # 다른 시도 명칭 (추천에서 배제)
    elif facility_sigungu != user_sigungu:
        score += 50 # 같은 시도 명칭 내 다른 시군구 명칭 (넓은 지역 일치)
    elif facility_eupmyeondong != user_eupmyeondong:
        score += 5 # 같은 시군구 명칭 내 다른 법정읍면동명칭 (근접 일치)
    else:
        score += 0 # 같은 읍면동명칭 (최고 일치)

    # 4. 주차 여부 (주차 가능여부) 점수 계산
    user_parking_pref = preferences.get('주차 가능여부')
    if user_parking_pref == 'Y':
        if facility_row['주차 가능여부'] == 'Y':
            score += 0 # 주차를 선호하고 시설도 주차 가능
        else:
            score += 10 # 주차를 선호하지만 시설은 주차 불가능 (감점)
    else: # user_parking_pref == 'N' (주차 불가능 선호) 또는 None (무관)
        score += 0 # 사용자가 주차를 선호하지 않거나 무관한 경우

    # 5. 실내/외 선호 (장소(실내) 여부, 장소(실외)여부) 점수 계산
    user_location_pref = preferences.get('장소 선호도')

    if user_location_pref == '실내':
        if facility_row['장소(실내) 여부'] == 'Y':
            score += 0 # 실내 선호하고 시설이 실내
        else:
            score += 1000 # 실내 선호하지만 시설이 실내가 아님 (추천에서 배제)
    elif user_location_pref == '실외':
        if facility_row['장소(실외)여부'] == 'Y':
            score += 0 # 실외 선호하고 시설이 실외
        else:
            score += 1000 # 실외 선호하지만 시설이 실외가 아님 (추천에서 배제)
    elif user_location_pref == '무관':
        score += 0 # 실내/외에 대해 무관한 경우

    return score

In [7]:
# 추천 프로그램 실행 로직 구현
def recommendation_program(df):
    print(f"\n--- {program_name} 시작 ---")

    while True:
        # 1. 사용자 선호도 입력받기
        preferences = get_user_preferences(df.copy())

        # 2. 각 시설에 대해 점수 계산
        # apply 메서드를 사용하여 각 행에 calculate_score 함수 적용
        df['score'] = df.apply(lambda row: calculate_score(row, preferences), axis=1)

        print("사용자 선호도에 따라 모든 시설의 점수 계산 완료.")

        # 3. 점수 100점 이하인 시설 필터링
        recommended_facilities = df[df['score'] <= 100]

        # 4. 'score' 컬럼 기준으로 오름차순 정렬
        recommended_facilities = recommended_facilities.sort_values(by='score').reset_index(drop=True)

        # 5. 상위 10개 시설 추출
        top_10_recommendations = recommended_facilities.head(10)

        # 6. 추천 목록 출력
        print("\n--- 추천 시설 목록 ---")
        if not top_10_recommendations.empty:
            for i, row in top_10_recommendations.iterrows():
                print(f"\n{i+1}. 시설명: {row['시설명']}")
                print(f"   카테고리: {row['카테고리3']}")
                print(f"   지번주소: {row['지번주소']}")
                print(f"   전화번호: {row['전화번호'] if pd.notna(row['전화번호']) else '정보 없음'}")
                print(f"   홈페이지: {row['홈페이지'] if pd.notna(row['홈페이지']) else '정보 없음'}")
                print(f"   운영시간: {row['운영시간'] if pd.notna(row['운영시간']) else '정보 없음'}")
                print(f"   주차 가능여부: {row['주차 가능여부']}")
                print(f"   장소(실내) 여부: {row['장소(실내) 여부']}")
                print(f"   장소(실외)여부: {row['장소(실외)여부']}")
                print(f"   추천 점수: {row['score']}")
        else:
            print("현재 조건에 맞는 추천 시설을 찾을 수 없습니다.")

        # 7. 재추천 또는 종료 선택
        while True:
            action = input("\n다시 추천받으시겠습니까? (R: 다시 추천, Q: 종료): ").strip().upper()
            if action == 'R':
                break # 다시 루프 시작
            elif action == 'Q':
                print("프로그램을 종료합니다. 감사합니다!")
                return # 함수 종료
            else:
                print("유효하지 않은 입력입니다. 'R' 또는 'Q'를 입력해주세요.")

In [11]:
1# 프로그램 실행
recommendation_program(df.copy())


--- 머랭이네 반려동물 시설 추천 시작 ---

--- 시설유형 선택 ---
선택 가능한 시설유형:
  1. 동물약국
  2. 미술관
  3. 카페
  4. 동물병원
  5. 반려동물용품
  6. 미용
  7. 문예회관
  8. 펜션
  9. 식당
  10. 여행지
  11. 위탁관리
  12. 박물관
  13. 호텔
원하는 시설유형을 입력하세요 (번호 또는 이름): 1

--- 도착 예정 시간 입력 ---
도착 예정 시간을 'HH:MM' 형식으로 입력하세요 (예: 09:30, 14:00): 12:11

--- 출발 위치 입력 ---

선택 가능한 시도 명칭:
  1. 경기도
  2. 전라남도
  3. 서울특별시
  4. 인천광역시
  5. 경상북도
  6. 대구광역시
  7. 세종특별자치시
  8. 전라북도
  9. 경상남도
  10. 강원도
  11. 부산광역시
  12. 광주광역시
  13. 대전광역시
  14. 충청북도
  15. 제주특별자치도
  16. 충청남도
  17. 울산광역시
시도 명칭을 입력하세요 (번호 또는 이름): 1

'경기도' 내 선택 가능한 시군구 명칭:
  1. 고양시 덕양구
  2. 안양시 만안구
  3. 수원시 팔달구
  4. 의정부시
  5. 오산시
  6. 성남시 중원구
  7. 광명시
  8. 수원시 권선구
  9. 남양주시
  10. 수원시 장안구
  11. 구리시
  12. 화성시
  13. 평택시
  14. 광주시
  15. 용인시 기흥구
  16. 용인시 수지구
  17. 부천시
  18. 안산시 단원구
  19. 양주시
  20. 하남시
  21. 시흥시
  22. 고양시 일산서구
  23. 성남시 분당구
  24. 수원시 영통구
  25. 이천시
  26. 김포시
  27. 안양시 동안구
  28. 성남시 수정구
  29. 안성시
  30. 동두천시
  31. 포천시
  32. 고양시 일산동구
  33. 안산시 상록구
  34. 여주시
  35. 파주시
  36. 양평군
  37. 가평군
  38. 군포시
 